In [1]:
import math
import os
import pickle
import re
import sys

import japanize_matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from datasets import Dataset
from transformers import AutoTokenizer
import openai

sys.path.append("../src")
pd.set_option("display.max_columns", None)
sns.set(font="IPAexGothic")
%matplotlib inline

## ■ データの読み込み

In [2]:
# データの読み込み
answers = pd.read_csv("Answers.csv")
questions = pd.read_csv("Questions.csv")
categories = pd.read_csv("Categories.csv")
ans2cat = pd.read_csv("Answer2Category.csv")
ans2tag = pd.read_csv("Answer2Tag.csv")

## ■ データの確認

### ・回答データ

In [3]:
answers

,AID,Text
0,A001,資料が見つからない場合は、以下の点を確認してください。<br><br><br>【受講生編】<...
1,A002,資料のアップロードやお知らせ作成時の電子メールでの通知の有無は、各授業の担当教員が設定できま...
2,A003,kibacoにはファイルへパスワードを設定する機能はありません。資料は受講生全員に開示されま...
3,A004,「資料」機能を使うことで可能です。詳細は<a href=http://www.comp.tm...
4,A005,科目に対応するコースへ学生を登録すれば、履修申請していない学生に対しても資料を配布できます。...
...,...,...
74,A075,年度が変わると前年度のコースの授業タブは非表示になりますが、マイページの授業一覧からアクセス...
75,A076,授業コースの「授業情報」に「機能を編集」ボタンがあります。この中のリストにチェックを入れるこ...
76,A077,インターネットに接続できれば、本学以外の場所でもkibacoを利用できます。kibacoのす...
77,A078,ログを見ることは出来ません。特別な事情がある場合には、システム管理室2（e-learning...


In [4]:
# キーの重複確認
answers["AID"].duplicated().sum()

0

In [5]:
# 一部項目名の変更
answers = answers.rename(columns={"Text": "answer"})

### ・質問データ

In [6]:
questions

,Text,AID
0,履修している授業で先生が資料をアップロードしているはずだが、コース上に資料が見当たらない。,A001
1,資料をマイページに置いたが、学生からは見えなかった。,A001
2,前期の科目の「資料」を学生から見られないようにするにはどうしたら良いか？,A001
3,アップロード済みの資料が見当たらない。,A001
4,先生の資料が見られない。,A001
...,...,...
422,推奨環境のブラウザは何か？,A079
423,どのOSを使えば良いの？,A079
424,どのブラウザなら使えるの？,A079
425,推奨環境を教えて。,A079


In [7]:
# 一部項目名を変更
questions = questions.rename(columns={" AID": "AID", "Text": "question"})
questions["AID"].nunique()

79

### ・カテゴリデータ

In [8]:
categories

,CID,Title
0,C001,資料
1,C002,課題
2,C003,テスト/アンケート
3,C004,コンテンツ
4,C005,アップロード
5,C006,登録・履修
6,C007,科目集約
7,C008,ログイン
8,C009,連絡
9,C010,"学生"""


In [9]:
ans2cat

,AID,CID
0,A001,C001
1,A002,C001
2,A003,C001
3,A004,C001
4,A005,C001
...,...,...
74,A075,C011
75,A076,C011
76,A077,C011
77,A078,C011


## ■ 前処理

In [10]:
questions.columns

Index(['question', 'AID'], dtype='object')

In [11]:
# 質問IDを付与
questions["QID"] = [f"Q{i:03}" for i in range(1, len(questions) + 1)]

# 回答データにカテゴリを付与しておく
answers = answers.merge(ans2cat, how="left", on="AID")

In [12]:
# 学習とテストの質問IDを分割しておく
from sklearn.model_selection import train_test_split

train_id, test_id = train_test_split(questions["QID"].to_list(), random_state=77)

In [13]:
## データセットの作成
# Tripletのpositiveの紐づけ
base_data = (questions.merge(answers, how="left", on="AID")
                    .rename(columns={"answer": "answer_pos", "CID": "CID_pos"}))

# Tripletのnegativeを一旦positive以外の回答を紐づけ作成
base_data = (base_data.merge(answers.rename(columns={"AID": "AID_neg"}), how="cross")
                      .rename(columns={"answer": "answer_neg", "CID": "CID_neg"}))
base_data = base_data.query("AID != AID_neg")

In [14]:
# positiveとnegativeのカテゴリが異なるデータを学習データとして使用
train = base_data.query(f"QID in {train_id} and CID_pos != CID_neg").sample(1000, random_state=7)

## ■ モデル定義

In [48]:
!pip install fugashi ipadic

  Obtaining dependency information for fugashi from https://files.pythonhosted.org/packages/63/9b/0fd0d9daea50204b469406a36a26bcd7c73d5eafb3a268cdd32d0e7759d9/fugashi-1.3.2-cp39-cp39-win_amd64.whl.metadata
     ---------------------------------------- 0.0/13.4 MB ? eta -:--:--
     ---------------------------------------- 0.0/13.4 MB 1.9 MB/s eta 0:00:07
     ---------------------------------------- 0.1/13.4 MB 1.6 MB/s eta 0:00:09
      --------------------------------------- 0.2/13.4 MB 1.5 MB/s eta 0:00:09
      --------------------------------------- 0.3/13.4 MB 1.4 MB/s eta 0:00:10
      --------------------------------------- 0.3/13.4 MB 1.6 MB/s eta 0:00:09
     - -------------------------------------- 0.4/13.4 MB 1.4 MB/s eta 0:00:10
     - -------------------------------------- 0.4/13.4 MB 1.4 MB/s eta 0:00:09
     - -------------------------------------- 0.5/13.4 MB 1.4 MB/s eta 0:00:10
     - -------------------------------------- 0.6/13.4 MB 1.4 MB/s eta 0:00:09
     - ----

In [15]:
from sentence_transformers import SentenceTransformer, SentencesDataset, InputExample, losses, models

# モデルの定義
bert = models.Transformer("sonoisa/sentence-bert-base-ja-mean-tokens-v2")
pooling = models.Pooling(bert.get_word_embedding_dimension())
model = SentenceTransformer(modules=[bert, pooling])

## ■ 学習前に推論

In [16]:
from sentence_transformers import evaluation

# テスト用のクエリ、検索コーパス、クエリに対する正解回答を準備
test_queries = questions.query(f"QID in {test_id}").set_index("QID")["question"].to_dict()
corpus = answers.set_index("AID")["answer"].to_dict()
relevant_docs = base_data[["QID", "AID"]].drop_duplicates().set_index("QID")["AID"].to_dict()

# MRR評価用のクラス（他の評価結果もでてくる）
evaluator = evaluation.InformationRetrievalEvaluator(test_queries, corpus, relevant_docs, mrr_at_k=[10])

In [ ]:
# 評価の実施と保存
_ = model.evaluate(evaluator, output_path="./before_finetune")

In [26]:
eval_result = pd.read_csv("./before_finetune/Information-Retrieval_evaluation_results.csv")
print(f'MRR: {eval_result["cos_sim-MRR@10"][0]:.3f}')

MRR: 0.435


・以下で手順を追ってMRRを算出し値が整合しているか確認

In [207]:
# あらかじめ検索文書の方をembeddingしておく
# 今回は簡単のためFAISS等のベクトルデータベースを使わず、リストでもつ
corpus_embeddings = model.encode(list(answers['answer']), convert_to_tensor=True)
corpus_embeddings.shape

torch.Size([79, 768])

In [208]:
from sentence_transformers import util, evaluation
import torch

# テスト用のクエリ、クエリに対する正解回答を準備
test_queries = questions.query(f"QID in {test_id}")["question"].to_list()
relevant_indices = (questions.query(f"QID in {test_id}")["AID"].str[1:].astype(int) - 1).to_list()

# 入力クエリの埋め込みとコサイン類似度算出
query_embeddings = model.encode(test_queries, convert_to_tensor=True)
cosine_scores = util.cos_sim(query_embeddings, corpus_embeddings)
top_results = torch.topk(cosine_scores, k=10)

# 評価指標MRRの算出
rank = np.where(top_results.indices.to("cpu").numpy() == np.array(relevant_indices).reshape(-1, 1))[1] + 1
mrr = (1 / rank).sum() / len(test_queries)
print(f"MRR: {mrr.3f}")

MRR: 0.43529891707461804


## ■ fine-tuning

In [17]:
from torch.utils.data import DataLoader

# 学習データと損失関数の作成
train_dataset = SentencesDataset(
    [InputExample(texts=[row["question"], row["answer_pos"], row["answer_neg"]]) 
     for index, row in train.iterrows()], model)
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=4)
train_loss = losses.TripletLoss(model=model)

# fine-tuningの実施
model.fit(
        train_objectives=[(train_dataloader, train_loss)],
        epochs=3,
        evaluator=evaluator,
        save_best_model=True,
        output_path="./sbert_finetuned",
        )

Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Iteration:   0%|          | 0/250 [00:00<?, ?it/s]

Iteration:   0%|          | 0/250 [00:00<?, ?it/s]

Iteration:   0%|          | 0/250 [00:00<?, ?it/s]

## ■ 評価

In [18]:
# 評価の実施と保存
_ = model.evaluate(evaluator, output_path="./after_finetune")

In [27]:
eval_result_finetuned = pd.read_csv("./after_finetune/Information-Retrieval_evaluation_results.csv")
print(f'MRR: {eval_result["cos_sim-MRR@10"][0]:.3f}')
print(f'MRR(finetuned): {eval_result_finetuned["cos_sim-MRR@10"][0]:.3f}')

MRR: 0.435
MRR(finetuned): 0.595
